<a href="https://colab.research.google.com/github/adarsh415/CNN_For_Image_Recog/blob/master/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras import metrics, datasets, layers, models, optimizers
import numpy as np

TensorFlow 2.x selected.


In [0]:
def load_data():
  (Xtrain, Ytrain), (Xtest, Ytest) = datasets.cifar10.load_data()
  Xtrain, Xtest = Xtrain.astype(np.float32)/255.0, Xtest.astype(np.float32)/255.0
  trainDS = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
  trainDS = trainDS.shuffle(50000).batch(128)
  testDS = tf.data.Dataset.from_tensor_slices((Xtest, Ytest))
  testDS = testDS.batch(64)

  return trainDS, testDS

In [0]:
class InceptionBlk(models.Model):
  def __init__(self,
               filter_1x1,
               filter_3x3,
               reduce_3x3,
               filter_5x5,
               reduce_5x5,
               pool_filter,
               kernel_init, bias_init):
    super(InceptionBlk, self).__init__(name='InceptionBlk')

    self.conv1x1 = layers.Conv2D(filter_1x1, kernel_size=1, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)
    self.conv3x3_reduce =  layers.Conv2D(reduce_3x3, kernel_size=1, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)
    self.conv3x3 = layers.Conv2D(filter_3x3, kernel_size=3, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)
    self.conv5x5_reduce = layers.Conv2D(reduce_5x5,kernel_size=1, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)
    self.conv5x5 = layers.Conv2D(filter_5x5, kernel_size=3, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)
    self.max5x5 = layers.MaxPool2D(pool_size=(5,5), strides=1, padding='same')
    self.conv1x1_max = layers.Conv2D(pool_filter, kernel_size=1, strides = 1, padding='same',activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)

  def call(self, x, training=None):

    x_1 = self.conv1x1(x)

    x_3 = self.conv3x3_reduce(x)
    x_3 = self.conv3x3(x_3)

    x_5 = self.conv5x5_reduce(x)
    x_5 = self.conv5x5(x_5)

    x_pool = self.max5x5(x)
    x_pool = self.conv1x1_max(x_pool)

    return layers.concatenate([x_1, x_3, x_5, x_pool])

In [0]:
class InceptionV3(models.Model):
  def __init__(self, kernel_init, bias_init, aux=3):
    super(InceptionV3, self).__init__(self)
    self.kernel_init = kernel_init
    self.bias_init = bias_init
    self.aux = aux


  def call(self, x, training=None):

    # first conv block
    block_1 = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu', kernel_initializer=self.kernel_init, bias_initializer=self.bias_init)(x)
    block_1 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(block_1)
    block_1 = layers.BatchNormalization()(block_1)

    # second conv block
    block_2 = layers.Conv2D(64, kernel_size=1, strides=1, padding='same', activation='relu', kernel_initializer=self.kernel_init, bias_initializer=self.bias_init)(x)
    block_2 = layers.Conv2D(192, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer=self.kernel_init, bias_initializer=self.bias_init)(x)
    block_2 = layers.BatchNormalization()(block_2)
    block_2 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(block_2)

    # inception block 1a
    inception_1a = InceptionBlk(filter_1x1=64,
                                filter_3x3=128,
                                reduce_3x3=96,
                                filter_5x5=32,
                                reduce_5x5=16,
                                pool_filter=32,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(block_2)
    # inception block 2a
    inception_2a = InceptionBlk(filter_1x1=128,
                                filter_3x3=192,
                                reduce_3x3=128,
                                filter_5x5=96,
                                reduce_5x5=32,
                                pool_filter=64,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_1a)

    # max pool layer
    pool_1 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(inception_2a)

    # inception block 3a
    inception_3a = InceptionBlk(filter_1x1=192,
                                filter_3x3=208,
                                reduce_3x3=96,
                                filter_5x5=48,
                                reduce_5x5=16,
                                pool_filter=64,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(pool_1)

    # first softmax layer
    soft_1 = layers.AveragePooling2D(pool_size=5, strides=3, padding='valid')(inception_3a)
    soft_1 = layers.Conv2D(128, kernel_size=1, strides=1, padding='same', activation='relu', kernel_initializer=self.kernel_init, bias_initializer=self.bias_init)(soft_1)
    soft_1 = layers.Flatten()(soft_1)
    soft_1 = layers.Dense(1024, activation='relu')(soft_1)
    soft_1 = layers.Dropout(0.7)(soft_1)
    soft_1 = layers.Dense(10, activation='softmax')(soft_1)
    

    #inception block 4a
    inception_4a = InceptionBlk(filter_1x1=160,
                                filter_3x3=224,
                                reduce_3x3=112,
                                filter_5x5=64,
                                reduce_5x5=24,
                                pool_filter=64,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_3a)

    #inception block 5a
    inception_5a = InceptionBlk(filter_1x1=128,
                                filter_3x3=256,
                                reduce_3x3=128,
                                filter_5x5=64,
                                reduce_5x5=24,
                                pool_filter=64,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_4a)

    #inception block 6a
    inception_6a = InceptionBlk(filter_1x1=112,
                                filter_3x3=288,
                                reduce_3x3=144,
                                filter_5x5=64,
                                reduce_5x5=32,
                                pool_filter=64,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_5a)


    #second softmax layer
    soft_2 = layers.AveragePooling2D(pool_size=5, strides=3, padding='valid')(inception_6a)
    soft_2 = layers.Conv2D(128, kernel_size=1, strides=1, padding='same', activation='relu', kernel_initializer=self.kernel_init, bias_initializer=self.bias_init)(soft_2)
    soft_2 = layers.Flatten()(soft_2)
    soft_2 = layers.Dense(1024, activation='relu')(soft_2)
    soft_2 = layers.Dropout(0.7)(soft_2)
    soft_2 = layers.Dense(10, activation='softmax')(soft_2)

    #inception block 7a
    inception_7a = InceptionBlk(filter_1x1=256,
                                filter_3x3=320,
                                reduce_3x3=160,
                                filter_5x5=128,
                                reduce_5x5=32,
                                pool_filter=128,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_6a)

    # max pool layer
    pool_2 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(inception_7a)
    
    #inception block 8a
    inception_8a = InceptionBlk(filter_1x1=256,
                                filter_3x3=320,
                                reduce_3x3=160,
                                filter_5x5=128,
                                reduce_5x5=32,
                                pool_filter=128,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(pool_2)
    
    #inception block 9a
    inception_9a = InceptionBlk(filter_1x1=384,
                                filter_3x3=384,
                                reduce_3x3=192,
                                filter_5x5=128,
                                reduce_5x5=48,
                                pool_filter=128,
                                kernel_init=self.kernel_init,
                                bias_init= self.bias_init)(inception_8a)

    #third softmax layer
    #soft_3 = layers.AveragePooling2D(pool_size=7, strides=1, padding='valid')(inception_9a)
    #soft_3 = layers.AveragePooling2D()(inception_9a)
    soft_3 = layers.GlobalAveragePooling2D()(inception_9a)
    soft_3 = layers.Dropout(0.4)(soft_3)
    soft_3 = layers.Flatten()(soft_3)
    soft_3 = layers.Dense(1024, activation='relu')(soft_3)
    soft_3 = layers.Dense(10, activation='softmax')(soft_3)

    if self.aux == 1:
      return soft_1
    elif self.aux == 2:
      return soft_2
    else:
    #return layers.concatenate([soft_1,soft_2, soft_3], axis=-1)
      return soft_3


In [0]:
def train():
    n_epoch = 50
    train_ds, test_ds = load_data()

    optimizer = optimizers.Adam(learning_rate=1e-3)

    kernel_init = k.initializers.he_normal()
    bias_init = k.initializers.he_normal()
    model1, model2, model3 = InceptionV3(kernel_init,bias_init,aux=1), InceptionV3(kernel_init,bias_init,aux=2), InceptionV3(kernel_init,bias_init)

    # acc = metrics.CategoricalAccuracy()
    acc = metrics.Accuracy()
    lossfn = tf.keras.losses.CategoricalCrossentropy()
    
    for epoch in range(n_epoch):
        losses = 0.0
        accuracy = 0.0
        #step = 0
        for step, (x,y) in enumerate(train_ds):
            #print(.shape)
            #step += 1
            # with tf.GradientTape() as tape1:
            #     logits1 = model1(x)
            #     loss1 = lossfn(tf.one_hot(y,10),logits1)
                
            # with tf.GradientTape() as tape2:
            #   logits2 = model2(x)
            #   loss2 = lossfn(tf.one_hot(y,10),logits2)

            with tf.GradientTape() as tape3:
              logits3 = model3(x)
              #print('logit shape', logits3.shape)
              loss3 = lossfn(tf.one_hot(y,10),logits3)
              #acc.update_state(tf.one_hot(y,10), logits3) this should be used with CategoricalAccuracy()
              acc.update_state(y, tf.argmax(logits3, axis=1, output_type=tf.int32))
              losses = loss3

            # grads1 = tape1.gradient(loss1, model1.trainable_variables)
            # grads1 = [tf.clip_by_norm(g, 15) for g in grads1]
            # optimizer.apply_gradients(zip(grads1, model1.trainable_variables))

            # grads2 = tape2.gradient(loss2, model2.trainable_variables)
            # grads2 = [tf.clip_by_norm(g, 15) for g in grads2]
            # optimizer.apply_gradients(zip(grads2, model2.trainable_variables))

            grads3 = tape3.gradient(loss3, model3.trainable_variables)
            grads3 = [tf.clip_by_norm(g, 15) for g in grads3]
            optimizer.apply_gradients(zip(grads3, model3.trainable_variables))

            
            if step%1000 == 0:
                #accuracy = compute_accuracy(logits, tf.cast(y, tf.int64))
                #print(f' accuracy in epoch {epoch} after steps {step} is: {accuracy.numpy()}')
                print(f' accuracy in epoch {epoch} after steps {step} is: {acc.result().numpy()}')
                acc.reset_states()
                #print(f' Loss in epoch {epoch} after steps {step} is: {losses}')
                print(f' Loss for main model3 in epoch {epoch} after steps {step} is: {losses.numpy()}')
                # print(f' Loss for auxialiary model2 in epoch {epoch} after steps {step} is: {loss2.numpy()}')
                # print(f' Loss for auxialiary model2 in epoch {epoch} after steps {step} is: {loss1.numpy()}')

        print(f'loss after epoch {epoch} is {losses.numpy()}')



    print(f'final training accuracy is {acc.result().numpy()}')
    acc.reset_states()


    #model.save('VGG16', save_format='tf')
    for step, (x, y) in enumerate(test_ds):
        logits = model3(x, training=False)
        acc.update_state(y, tf.argmax(logits, axis=1, output_type=tf.int32))
        #acc.update_state(tf.one_hot(y,10), logits)
        # if step%1000 == 0:
        #     #y = tf.squeeze(y, axis=1)
        #     #accuracy = compute_accuracy(logits,tf.cast(y, tf.int64))
            
    print(f'test accuracy is {acc.result().numpy()}')
    acc.reset_states()


In [0]:
train()

 accuracy in epoch 0 after steps 0 is: 0.0703125
 Loss for main model3 in epoch 0 after steps 0 is: 3.434532880783081
loss after epoch 0 is 2.5541484355926514
 accuracy in epoch 1 after steps 0 is: 0.10106000304222107
 Loss for main model3 in epoch 1 after steps 0 is: 2.5021181106567383
loss after epoch 1 is 3.022416353225708
 accuracy in epoch 2 after steps 0 is: 0.09882000088691711
 Loss for main model3 in epoch 2 after steps 0 is: 2.5204873085021973
loss after epoch 2 is 2.522526741027832
 accuracy in epoch 3 after steps 0 is: 0.09892000257968903
 Loss for main model3 in epoch 3 after steps 0 is: 2.7066874504089355
loss after epoch 3 is 2.4204823970794678
 accuracy in epoch 4 after steps 0 is: 0.09950000047683716
 Loss for main model3 in epoch 4 after steps 0 is: 3.10794734954834
loss after epoch 4 is 2.690838575363159
 accuracy in epoch 5 after steps 0 is: 0.10062000155448914
 Loss for main model3 in epoch 5 after steps 0 is: 2.4790072441101074
loss after epoch 5 is 2.8630917072296